# AOC 2018

Welcome to the Advent of Code 2018 !

## Basic configuration



In [ ]:
! pip install -U advent-of-code-data numpy pandas

In [ ]:
import os

os.environ['AOC_SESSION'] = open('session.txt').read().strip()

In [ ]:
from aocd.models import Puzzle
from collections import Counter, defaultdict
from itertools import chain, product
from numpy.typing import ArrayLike
from pprint import pprint
from queue import PriorityQueue
from statistics import median
from typing import Callable, Dict, Iterator, Mapping, Tuple
import copy
import functools
import math
import numpy as np
import pandas as pd
import operator
import re
import sys

In [ ]:
def import_ints(input_data:str, as_nparray:bool=False) -> Iterator[int]:
    """Transform the input string into a list of integers."""
    if "\n" in input_data:
        if as_nparray:
            return np.array([list(x) for x in input_data.splitlines()], dtype=int)
        else:
            return [int(x) for x in input_data.splitlines()]
    
    else:
        return [int(x) for x in input_data.split(",")]

# Problems

## Day 1

http://adventofcode.com/2018/day/1

In [ ]:
puzzle = Puzzle(year=2018, day=1)

__Part 1__

In [ ]:
problem = puzzle.input_data.replace("\n", "")
eval(f"0{problem}")
# puzzle.anwser_a = eval(f"0{problem}")

__Part 2__

In [ ]:
inputs = puzzle.input_data.splitlines()
cont, value, values = True, 0, {0}

while cont:
    for op in inputs:
        if op[0] == "+":
            value += int(op[1:])
        else:
            value -= int(op[1:])

        if value in values:
            puzzle.anwser_b = value
            cont = False
            break
        else:
            values.add(value)
pprint(value)

## Day 2

http://adventofcode.com/2018/day/2

In [ ]:
puzzle = Puzzle(year=2018, day=2)

__Part 1__

In [ ]:
count2, count3 = 0, 0

for line in puzzle.input_data.splitlines():
    count2 += int(any(map(lambda x: x == 2, Counter(line).values())))
    count3 += int(any(map(lambda x: x == 3, Counter(line).values())))

puzzle.answer_a = count2 * count3
pprint(count2 * count3)

__Part 2__

In [ ]:
box_ids, common = [], ""

for line in puzzle.input_data.splitlines():    
    for old_box in box_ids:        
        diff = sum(map(lambda x: int(x[0] != x[1]), zip(line, old_box)))
        
        if diff == 1:
            common = "".join([x[0] for x in filter(lambda x: x[0] == x[1], zip(line, old_box))])
            break
                   
    if common:
        break
            
        
    box_ids.append(line)

# puzzle.answer_b = common
pprint(common)

## Day 3

http://adventofcode.com/2018/day/3

In [ ]:
puzzle = Puzzle(year=2018, day=3)

__Part 1__

In [ ]:
interpreter = re.compile("#(\d+)\s@\s(\d+),(\d+):\s(\d+)x(\d+)")
patch = Counter()

for line in puzzle.input_data.splitlines():
    claim_id, col_idx, row_idx, width, height = map(int, interpreter.search(line).groups())
    for x in range(row_idx, row_idx + height):
        for y in range(col_idx, col_idx + width):
            patch[(x, y)] += 1

In [ ]:
patch_values = pd.DataFrame(patch.values())
puzzle.answer_a = int(patch_values[patch_values > 1].count())

__Part 2__

In [ ]:
for line in puzzle.input_data.splitlines():
    overlap = len(patch) == 0  # says True for the first line 

    claim_id, col_idx, row_idx, width, height = map(int, interpreter.search(line).groups())
    for x in range(row_idx, row_idx + height):
        for y in range(col_idx, col_idx + width):
            if not overlap and patch[(x, y)] > 1:
                overlap = True
    if not overlap:
        puzzle.answer_b = claim_id
        pprint(claim_id)
        break

## Day 4

http://adventofcode.com/2018/day/4

In [ ]:
puzzle = Puzzle(year=2018, day=4)

In [ ]:
re_moment = re.compile("\[\d+-(\d+)-(\d+)\s(\d+):(\d+)\]")

logs = sorted(puzzle.input_data.splitlines())
for log in logs:
    moment, action = log.split(" ", 1)
    
    if "Guard" in action:

__Part 1__

__Part 2__

## Day 5

http://adventofcode.com/2018/day/5

In [ ]:
puzzle = Puzzle(year=2018, day=5)

__Part 1__

__Part 2__

## Day 6

http://adventofcode.com/2018/day/6

In [ ]:
puzzle = Puzzle(year=2018, day=6)

In [ ]:
points = list(map(lambda line: list(map(int, line.split(','))), puzzle.input_data.splitlines()))
# points = [[1, 1], [1, 6], [8, 3], [3, 4], [5, 5], [8, 9]]  # Example

min_x = min(map(lambda p: p[0], points))
min_y = min(map(lambda p: p[1], points))
max_x = max(map(lambda p: p[0], points))
max_y = max(map(lambda p: p[1], points))

nb_points = len(points)

# points = list(map(lambda p: [p[0] - min_x, p[1] - min_y], points))
# max_x -= min_x
# max_y -= min_y
# min_x = min_y = 0

__Part 1__

In [ ]:
def distance(from_point, to_point):
    return abs(from_point[0] - to_point[0]) + abs(from_point[1] - to_point[1])

In [ ]:
def store_grid():
    grid_lines = []
    for x_idx in range(min_x, max_x + 1):
        line = ""
        for y_idx in range(min_y, max_y + 1):
            if len(attribution[x_idx - min_x][y_idx - min_y][1]) == 1:
                ref = "A" if attribution[x_idx - min_x][y_idx - min_y][0] == 0 else "a"
                line += chr(ord(ref) + attribution[x_idx - min_x][y_idx - min_y][1][0])
            else:
                line += '.'
        grid_lines += [line]
    return grid_lines

def display_grid(lines=None):
    if lines is None:
        lines = store_grid()

    for line in lines:
        print(line)

In [ ]:
# Version 1
attribution = []

for x_idx in range(min_x, max_x + 1):
    row = []
    for y_idx in range(min_y, max_y + 1):
        elem = []
        for p_idx in range(nb_points):
            d = distance(points[p_idx], [x_idx, y_idx])
            if not elem or d < elem[0]:
                elem = [d, [p_idx]]
            elif d == elem[0]:
                elem = [d, elem[1] + [p_idx]]
        row += [elem]
    attribution += [row]

grid_lines = store_grid()
borders = grid_lines[0] + grid_lines[-1] + ''.join(map(lambda x: x[0], grid_lines)) + ''.join(map(lambda x: x[-1], grid_lines))
excluded_candidates = set(x for x in borders.lower())

counter = Counter([y.lower() for x in grid_lines for y in x])

for candidate in excluded_candidates:
    del counter[candidate]

print(f"Answer 1: {counter.most_common(1)}")
puzzle.answer_a = counter.most_common(1)[0][1]

# display_grid(lines=grid_lines)

In [ ]:
# Version 2
counter = Counter()
excluded_candidates = set()

for x_idx in range(min_x, max_x + 1):
    for y_idx in range(min_y, max_y + 1):
        scores = []
        for p_idx in range(nb_points):
            scores += [distance(points[p_idx], [x_idx, y_idx])]

        scores = np.array(scores)
        candidates = np.argwhere(scores == scores.min())
        
        if candidates.size == 1:
            counter[candidates[0][0]] += 1

            if x_idx in [min_x, max_x] or y_idx in [min_y, max_y]:
                excluded_candidates.add(candidates[0][0])
        
for cand in excluded_candidates:
    del counter[cand]

print(f"Answer 1: {counter.most_common(1)}")
puzzle.answer_a = counter.most_common(1)[0][1]

__Part 2__

In [ ]:
region_size = 0

for x_idx in range(min_x, max_x + 1):
    for y_idx in range(min_y, max_y + 1):
        score = 0
        for p_idx in range(nb_points):
            score += distance(points[p_idx], [x_idx, y_idx])
            
        if score < 10000:
            region_size += 1

print(f"Answer 2: {region_size}")
puzzle.answer_b = region_size

## Day 7

http://adventofcode.com/2018/day/7

In [ ]:
puzzle = Puzzle(year=2018, day=7)

__Part 1__

__Part 2__

## Day 8

http://adventofcode.com/2019/day/8

In [ ]:
puzzle = Puzzle(year=2018, day=8)

__Part 1__

__Part 2__

## Day 9

http://adventofcode.com/2018/day/9

In [ ]:
puzzle = Puzzle(year=2018, day=9)

__Part 1__

__Part 2__

## Day 10

http://adventofcode.com/2018/day/10

In [ ]:
puzzle = Puzzle(year=2018, day=10)

__Part 1__

__Part 2__

## Day 11

http://adventofcode.com/2018/day/11

In [ ]:
puzzle = Puzzle(year=2018, day=11)

__Part 1__

__Part 2__

## Day 12

http://adventofcode.com/2018/day/12

In [ ]:
puzzle = Puzzle(year=2018, day=12)

__Part 1__

__Part 2__

## Day 13

http://adventofcode.com/2018/day/13

In [ ]:
puzzle = Puzzle(year=2018, day=13)

__Part 1__

__Part 2__

## Day 14

http://adventofcode.com/2018/day/14

In [ ]:
puzzle = Puzzle(year=2018, day=14)

__Part 1__

__Part 2__

## Day 15

http://adventofcode.com/2018/day/15

In [ ]:
puzzle = Puzzle(year=2018, day=15)

__Part 1__

__Part 2__

## Day 16

http://adventofcode.com/2018/day/16

In [ ]:
puzzle = Puzzle(year=2018, day=16)

__Part 1__

__Part 2__

## Day 17

http://adventofcode.com/2018/day/17

In [ ]:
puzzle = Puzzle(year=2018, day=17)

__Part 1__

__Part 2__

## Day 18

http://adventofcode.com/2018/day/18

In [ ]:
puzzle = Puzzle(year=2018, day=18)

__Part 1__

__Part 2__

## Day 19

http://adventofcode.com/2018/day/19

In [ ]:
puzzle = Puzzle(year=2018, day=19)

__Part 1__

__Part 2__

## Day 20

http://adventofcode.com/2018/day/20

In [ ]:
puzzle = Puzzle(year=2018, day=20)

__Part 1__

__Part 2__

## Day 21

http://adventofcode.com/2018/day/21

In [ ]:
puzzle = Puzzle(year=2018, day=21)

__Part 1__

__Part 2__

## Day 22

http://adventofcode.com/2018/day/22

In [ ]:
puzzle = Puzzle(year=2018, day=22)

__Part 1__

__Part 2__

## Day 23

http://adventofcode.com/2018/day/23

In [ ]:
puzzle = Puzzle(year=2018, day=23)

__Part 1__

__Part 2__

## Day 24

http://adventofcode.com/2018/day/24

In [ ]:
puzzle = Puzzle(year=2018, day=24)

__Part 1__

__Part 2__

## Day 25

http://adventofcode.com/2018/day/25

In [ ]:
puzzle = Puzzle(year=2018, day=25)

__Part 1__

__Part 2__